In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install osmnx

In [ ]:
train_df = pd.read_csv('/kaggle/input/uhi-index-data/Training_data_uhi_index_UHI2025-v2.csv')
test_df = pd.read_csv('/kaggle/input/test-df-ey-open-science/Submission_template_UHI2025-v2.csv')

In [ ]:
import osmnx as ox
from shapely import geometry as geom
import networkx as nx
# Download water bodies for NYC (polygons)
def get_osmnx_features(aoi_polygon):
    """Fetch comprehensive urban features from OSMnx limited to the given polygon."""
    print("Fetching comprehensive OSMnx features for AOI...")
    tags = {
        'building': True,
        'highway': True,
        'natural': 'water',
        'leisure': 'park',
    }
    gdf = ox.features_from_polygon(aoi_polygon, tags=tags)
    print("Fetched OSMnx features for AOI.")
    return gdf
water = ox.features_from_place("New York City, USA", tags={"natural": "water"})

# Download parks (replace with NYC OpenData link for higher accuracy)
parks = ox.features_from_place("New York City, USA", tags={"leisure": "park"})
nyc_bbox = (-74.01, 40.75, -73.86, 40.88)  # in EPSG:4326
nyc_poly = geom.box(*nyc_bbox)
G = ox.graph_from_polygon(nyc_poly, network_type='drive')
roads = ox.graph_to_gdfs(G, nodes=False, edges=True).to_crs("EPSG:2263")
nodes = ox.graph_to_gdfs(G, nodes=True, edges=False).to_crs("EPSG:2263")
closeness_dict = nx.closeness_centrality(G)
nodes["closeness"] = nodes.index.map(closeness_dict)
# nyc_features = get_osmnx_features(nyc_poly).to_crs("EPSG:2263")
# buildings = nyc_features[nyc_features["building"].notna()].copy()

In [ ]:
from shapely.geometry import Point
import geopandas as gpd

# Convert train/test coordinates to GeoDataFrame
train_geometry = [Point(lon, lat) for lon, lat in zip(train_df["Longitude"], train_df["Latitude"])]
test_geometry = [Point(lon, lat) for lon, lat in zip(test_df["Longitude"], test_df["Latitude"])]
train_gdf = gpd.GeoDataFrame(train_df, geometry=train_geometry, crs="EPSG:4326")
train_gdf = train_gdf.to_crs('EPSG:2263')
test_gdf = gpd.GeoDataFrame(test_df, geometry=test_geometry, crs="EPSG:4326")
test_gdf = test_gdf.to_crs('EPSG:2263')
# # Compute distance to nearest water body (meters)
# train_gdf["distance_to_water"] = train_gdf.geometry.apply(
#     lambda x: water.geometry.distance(x).min()
# )
# test_gdf["distance_to_water"] = test_gdf.geometry.apply(
#     lambda x: water.geometry.distance(x).min()
# )

# # Compute distance to nearest park
# train_gdf["distance_to_park"] = train_gdf.geometry.apply(
#     lambda x: parks.geometry.distance(x).min()
# )
# test_gdf["distance_to_park"] = test_gdf.geometry.apply(
#     lambda x: parks.geometry.distance(x).min()
# )
# train_gdf = train_gdf.to_crs('EPSG:4326')
# test_gdf  = test_gdf.to_crs('EPSG:4326')

In [ ]:
import networkx as nx
import numpy as np
import geopandas as gpd
import osmnx as ox
from shapely import geometry as geom
import pandas as pd

def compute_all_spatial_features(gdf_points, water, parks, roads=None, nodes=None, buffer_m=10000):
    """
    Computes spatial features for each point:
    
      Baseline features:
        - distance_to_water: minimum distance to any water body.
        - distance_to_park: minimum distance to any park.
        
      Buffer‐derived road network metrics (using OSMnx):
        - avg_street_length_osm: mean road segment length within the buffer.
        - road_density_osm: total road length within the buffer divided by the buffer area.
        - node_density_osm: count of road nodes per buffer area.
        - road_segment_count_osm: count of road segments within the buffer.
        - road_length_std_osm: standard deviation of road segment lengths within the buffer.
      
      Additional Crucial Features:
        - avg_closeness_centrality_osm: average closeness centrality of road network nodes within the buffer.
        - vegetation_ratio_osm: fraction of the buffer area covered by parks.
    
    All spatial operations are done in EPSG:2263 for meter‐based calculations;
    the final output is reprojected back to EPSG:4326.
    """
    print("Reprojecting inputs to EPSG:2263...")
    # Reproject inputs for meter-based calculations.
    gdf_points = gdf_points.to_crs("EPSG:2263").copy()
    water = water.to_crs("EPSG:2263")
    parks = parks.to_crs("EPSG:2263")
    
    print("Computing baseline features...")
    # Baseline features.
    gdf_points["distance_to_water"] = gdf_points.geometry.apply(lambda x: water.geometry.distance(x).min())
    gdf_points["distance_to_park"] = gdf_points.geometry.apply(lambda x: parks.geometry.distance(x).min())
    
    print("Creating buffer for each point...")
    # Create a buffer around each point.
    gdf_points["buffer"] = gdf_points.geometry.buffer(buffer_m)
    
    print("Defining NYC polygon...")
    # Define NYC polygon for data downloads.
    nyc_bbox = (-74.01, 40.75, -73.86, 40.88)  # in EPSG:4326
    nyc_poly = geom.box(*nyc_bbox)
    
    # Download road network and nodes if not provided.
    if roads is None or nodes is None:
        print("Downloading road network and nodes...")
        G = ox.graph_from_polygon(nyc_poly, network_type='drive')
        roads = ox.graph_to_gdfs(G, nodes=False, edges=True).to_crs("EPSG:2263")
        nodes = ox.graph_to_gdfs(G, nodes=True, edges=False).to_crs("EPSG:2263")
        print("Computing closeness centrality for nodes...")
        closeness_dict = nx.closeness_centrality(G)
        nodes["closeness"] = nodes.index.map(closeness_dict)
    else:
        print("Using pre-downloaded road network and nodes.")
    
    print("Performing spatial joins...")
    # Use the buffer geometry for spatial joins.
    buffer_gdf = gdf_points.copy().set_geometry("buffer")
    
    # Join with roads (edges).
    joined_edges = gpd.sjoin(buffer_gdf, roads, how="left", predicate="intersects")
    # Join with nodes.
    joined_nodes = gpd.sjoin(buffer_gdf, nodes, how="left", predicate="contains")
    # Join with parks (for vegetation cover).
    buffer_gdf["buffer_geom"] = buffer_gdf.geometry  # preserve buffer geometry.
    joined_parks = gpd.sjoin(buffer_gdf, parks, how="left", predicate="intersects")
    # Compute intersection area between each buffer and the park geometry.
    joined_parks["intersection_area"] = joined_parks.apply(
        lambda row: row["buffer_geom"].intersection(row["geometry"]).area if row["geometry"] is not None else 0, axis=1)
    
    print("Computing road network metrics...")
    metrics = pd.DataFrame(index=gdf_points.index)
    
    # Average street length within the buffer.
    avg_street_length = joined_edges.groupby(joined_edges.index)["length"].mean()
    metrics["avg_street_length_osm"] = avg_street_length.reindex(gdf_points.index, fill_value=0)
    
    # Road density: total road length divided by buffer area.
    total_road_length = joined_edges.groupby(joined_edges.index)["length"].sum()
    metrics["road_density_osm"] = total_road_length.reindex(gdf_points.index, fill_value=0) / gdf_points["buffer"].area
    
    # Node density: count of nodes per buffer area.
    node_count = joined_nodes.groupby(joined_nodes.index).size()
    metrics["node_density_osm"] = node_count.reindex(gdf_points.index, fill_value=0) / gdf_points["buffer"].area
    
    # Road segment count: count of road segments in the buffer.
    road_segment_count = joined_edges.groupby(joined_edges.index)["length"].count()
    metrics["road_segment_count_osm"] = road_segment_count.reindex(gdf_points.index, fill_value=0)
    
    # Road length standard deviation within the buffer.
    road_length_std = joined_edges.groupby(joined_edges.index)["length"].std()
    metrics["road_length_std_osm"] = road_length_std.reindex(gdf_points.index, fill_value=0)
    
    print("Computing additional crucial features...")
    # Additional features.
    # Average closeness centrality from nodes within the buffer.
    avg_closeness = joined_nodes.groupby(joined_nodes.index)["closeness"].mean()
    metrics["avg_closeness_centrality_osm"] = avg_closeness.reindex(gdf_points.index, fill_value=0)
    
    # Vegetation ratio: fraction of the buffer area covered by parks.
    # park_area = joined_parks.groupby(joined_parks.index)["intersection_area"].sum()
    # vegetation_ratio = park_area.reindex(gdf_points.index, fill_value=0) / gdf_points["buffer"].area
    # metrics["vegetation_ratio_osm"] = vegetation_ratio
    
    print("Merging metrics with original data...")
    # Merge computed metrics with the original GeoDataFrame.
    gdf_points = gdf_points.join(metrics)
    
    print("Dropping temporary buffer column and reprojecting to EPSG:4326...")
    # Remove the temporary buffer column and reproject the result.
    gdf_points.drop(columns=["buffer"], inplace=True)
    return gdf_points.to_crs("EPSG:4326")

# =============================================================================
# Example usage:
# Precompute the expensive datasets once and then pass them in to save computation.
# Assume train_gdf and test_gdf are GeoDataFrames (with coordinates in EPSG:4326),
# and water and parks are GeoDataFrames in EPSG:4326.
# =============================================================================
# print("Precomputing shared data (road network and nodes)...")
# nyc_bbox = (-74.01, 40.75, -73.86, 40.88)
# nyc_poly = geom.box(*nyc_bbox)

# print("Downloading road network and nodes for shared data...")
# G = ox.graph_from_polygon(nyc_poly, network_type='drive')
# roads_shared = ox.graph_to_gdfs(G, nodes=False, edges=True).to_crs("EPSG:2263")
# nodes_shared = ox.graph_to_gdfs(G, nodes=True, edges=False).to_crs("EPSG:2263")
# closeness_dict = nx.closeness_centrality(G)
# nodes_shared["closeness"] = nodes_shared.index.map(closeness_dict)

# print("Sample of computed features:")
# print(train_final[["distance_to_water", "distance_to_park",
#                     "avg_street_length_osm", "road_density_osm",
#                     "node_density_osm"]].head())


In [ ]:
# input_cols = [col for col in train_final.columns if col!='Longitude' and col!='Latitude'
#              and col!='datetime' and col!='geometry' and col!='UHI Index']
# input_cols

In [ ]:
def compute_features(train, test, water, parks, roads, nodes, buffer_list):
    """Processes multiple buffers into unified DataFrames with _[buffer]m suffixes"""
    
    # Initialize merged DataFrames with core columns
    merged_train = train[['Longitude', 'Latitude', 'datetime', 'UHI Index', 'geometry']].copy()
    merged_test = test[['Longitude', 'Latitude', 'geometry']].copy()  # Removed UHI Index if test doesn't have it

    for buffer in buffer_list:
        print(f'Processing buffer: {buffer}m')
        
        # Compute features for current buffer
        train_buffer = compute_all_spatial_features(train, water, parks, roads, nodes, buffer)
        test_buffer = compute_all_spatial_features(test, water, parks, roads, nodes, buffer)
        
        # Identify feature columns (exclude locators/target)
        feature_cols = [col for col in train_buffer.columns 
                        if col not in ['Longitude', 'Latitude', 'datetime', 'UHI Index', 'geometry']]
        
        # Add buffer suffix and merge
        merged_train = merged_train.join(
            train_buffer[feature_cols].add_suffix(f'_{buffer}m'))
        merged_test = merged_test.join(
            test_buffer[feature_cols].add_suffix(f'_{buffer}m'))
    
    # Drop geometry if not needed in final output
    merged_train = merged_train.drop(columns='geometry')
    merged_test = merged_test.drop(columns='geometry')
    
    # Save final unified datasets
    merged_train.to_csv('train_unified_buffers.csv', index=False)
    merged_test.to_csv('test_unified_buffers.csv', index=False)
    
    return merged_train, merged_test


buffer_list = [500, 1000, 1500,
               2000, 2500, 3000, 3500,
               4000, 4500, 5000, 6000, 7000, 7500, 8000,
               8500, 9000, 9500, 10000, 10500, 11000] 
train_full, test_full = compute_features(train_gdf, test_gdf, water, parks, roads, nodes, buffer_list)

In [ ]:
test_full.head(4)

In [ ]:
# train_final_1000 = compute_all_spatial_features(train_gdf, water, parks,
#                                            roads=roads, nodes=nodes, buffer_m=1000)
# test_final_1000  = compute_all_spatial_features(test_gdf, water, parks,
#                                           roads=roads, nodes=nodes, buffer_m=1000)
# train_final_100[input_cols].to_csv()

In [ ]:
# input_cols = ['distance_to_water', 'distance_to_park', 'avg_street_length_osm', 'road_density_osm', 'node_density_osm']

In [ ]:
# import xgboost as xgb
# xgb_reg = xgb.XGBRegressor()
# target = train_final['UHI Index']
# xgb_reg.fit(train_final[input_cols], target)

In [ ]:
# preds = xgb_reg.predict(test_final[input_cols])

In [ ]:
# test_df['UHI Index'] = preds
# test_df.to_csv('submission_osmnx_features.csv', index=False)

In [ ]:
# train_final[input_cols].to_csv('train_osmnx_milked.csv', index=False)
# test_final[input_cols].to_csv('test_osmnx_milked.csv'  , index=False  )

In [ ]:
# cols_to_save = ['distance_to_water', 'distance_to_park']
# train_gdf[cols_to_save].to_csv('train_dis.csv', index=False)
# test_gdf[cols_to_save].to_csv('test_dis.csv', index=False)

In [ ]:
# import shapely.geometry as geom
# import geopandas as gpd
# def compute_road_density(gdf_points, buffer_m=1500):
#     """
#     Computes road density (total road length per buffer area) for each point in gdf_points.
#     Instead of using the convex hull of gdf_points, we use a fixed bounding polygon
#     for NYC.
#     """
#     # Define a fixed bounding box for NYC in (min_lon, min_lat, max_lon, max_lat)
#     nyc_bbox = (-74.01, 40.75, -73.86, 40.88)
#     # Create a polygon from the bounding box (in lon, lat order, i.e. EPSG:4326)
#     nyc_poly = geom.box(*nyc_bbox)
    
#     # Download the road network within this polygon (driveable roads)
#     G = ox.graph_from_polygon(nyc_poly, network_type='drive')
#     roads = ox.graph_to_gdfs(G, nodes=False, edges=True)
#     # Reproject roads to EPSG:2263 (meters)
#     roads = roads.to_crs("EPSG:2263")
    
#     # Compute road density per point:
#     # Buffer each point by buffer_m meters.
#     gdf_points = gdf_points.copy()
#     gdf_points["buffer"] = gdf_points.geometry.buffer(buffer_m)
    
#     # Spatial join: intersect each buffer with the road network.
#     joined = gpd.sjoin(gdf_points.set_geometry("buffer"), roads, how="left", predicate="intersects")
    
#     # For each point, sum the 'length' of road segments that intersect.
#     # (Ensure that the roads GeoDataFrame has a 'length' column;
#     # if not, compute it using roads.geometry.length)
#     if "length" not in roads.columns:
#         roads["length"] = roads.geometry.length

#     road_length = joined.groupby(joined.index)["length"].sum()
    
#     # Calculate road density = total road length / area of the buffer.
#     gdf_points["road_density"] = road_length / gdf_points["buffer"].area
#     gdf_points["road_density"] = gdf_points["road_density"].fillna(0)
#     gdf_points.drop(columns=["buffer"], inplace=True)
    
#     return gdf_points


# # Uncomment if you want to add road density:
# train_gdf_rd = compute_road_density(train_gdf, buffer_m=1500)
# test_gdf_rd  = compute_road_density(test_gdf, buffer_m=1500)

In [ ]:
# def compute_osmnx_metrics(gdf_points, buffer_m=1500):
#     """
#     For each point in gdf_points (in EPSG:2263), creates a buffer of buffer_m meters
#     and computes:
#       - avg_street_length: mean road segment length in the buffer
#       - road_density: total road length per buffer area
#       - node_density: count of road nodes per buffer area
#       - avg_node_degree: approximate connectivity, 2*(# edges)/(# nodes)
#     Returns gdf_points with these new metrics, with suffix '_osm'.
#     """
#     # Fixed bounding polygon for NYC (in EPSG:4326), then reproject to EPSG:2263.
#     nyc_bbox = (-74.01, 40.75, -73.86, 40.88)
#     nyc_poly = geom.box(*nyc_bbox)
#     nyc_poly_gdf = gpd.GeoDataFrame({'geometry': [nyc_poly]}, crs="EPSG:4326").to_crs("EPSG:2263")
    
#     # Download road network for NYC (driveable network)
#     G = ox.graph_from_polygon(nyc_poly, network_type='drive')
#     roads = ox.graph_to_gdfs(G, nodes=False, edges=True).to_crs("EPSG:2263")
#     nodes = ox.graph_to_gdfs(G, nodes=True, edges=False).to_crs("EPSG:2263")
    
#     # Create a buffer around each point
#     gdf_points = gdf_points.copy()
#     gdf_points["buffer"] = gdf_points.geometry.buffer(buffer_m)
    
#     # Spatial join with roads (edges)
#     joined_edges = gpd.sjoin(gdf_points.set_geometry("buffer"), roads, how="left", predicate="intersects")
#     # Spatial join with nodes
#     joined_nodes = gpd.sjoin(gdf_points.set_geometry("buffer"), nodes, how="left", predicate="contains")
    
#     # Compute metrics per point in a new DataFrame
#     metrics = pd.DataFrame(index=gdf_points.index)
    
#     # 1. Average Street Length
#     avg_street_length = joined_edges.groupby(joined_edges.index)["length"].mean()
#     metrics["avg_street_length"] = avg_street_length.reindex(gdf_points.index, fill_value=0)
    
#     # 2. Road Density
#     total_road_length = joined_edges.groupby(joined_edges.index)["length"].sum()
#     metrics["road_density"] = total_road_length.reindex(gdf_points.index, fill_value=0) / gdf_points["buffer"].area
    
#     # 3. Node Density
#     node_count = joined_nodes.groupby(joined_nodes.index).size()
#     metrics["node_density"] = node_count.reindex(gdf_points.index, fill_value=0) / gdf_points["buffer"].area
    
#     # 4. Connectivity: approximate as 2 * (# unique edges) / (# unique nodes)
#     # unique_edges = joined_edges.groupby(joined_edges.index).apply(lambda df: df.index.nunique())
#     # unique_nodes = joined_nodes.groupby(joined_nodes.index).apply(lambda df: df.index.nunique())
#     # connectivity = 2 * unique_edges / unique_nodes.replace(0, np.nan)
#     # connectivity = connectivity.fillna(0)
#     # metrics["avg_node_degree"] = connectivity.reindex(gdf_points.index, fill_value=0)
    
#     # Add a suffix to avoid overlap
#     metrics = metrics.add_suffix("_osm")
    
#     # Remove the temporary buffer column and join metrics
#     gdf_points = gdf_points.drop(columns=["buffer"])
#     gdf_points = gdf_points.join(metrics)
    
#     return gdf_points

# # Example usage:
# train_gdf_osmnx_metrics = compute_osmnx_metrics(train_gdf, buffer_m=1500)
# test_gdf_osmnx_metrics  = compute_osmnx_metrics(test_gdf , buffer_m=1500)

In [ ]:
# test_gdf_osmnx_metrics.nunique()

In [ ]:
# def compute_intersection_density(gdf_points, buffer_m=1500):
#     # Define a fixed bounding box for NYC (EPSG:4326) and create a polygon
#     nyc_bbox = (-74.01, 40.75, -73.86, 40.88)
#     nyc_poly = geom.box(*nyc_bbox)
    
#     # Download the road network nodes (driveable network)
#     nodes = ox.graph_to_gdfs(ox.graph_from_polygon(nyc_poly, network_type='drive'),
#                               nodes=True, edges=False)
#     nodes = nodes.to_crs("EPSG:2263")
    
#     # Buffer each point
#     gdf_points = gdf_points.copy()
#     gdf_points["buffer"] = gdf_points.geometry.buffer(buffer_m)
    
#     # Spatial join: assign nodes (intersections) to each buffered point
#     joined = gpd.sjoin(gdf_points.set_geometry("buffer"), nodes, how="left", predicate="contains")
    
#     # Count intersections per point, and calculate density (intersections per square meter)
#     intersection_counts = joined.groupby(joined.index).size()
#     gdf_points["intersection_density"] = intersection_counts.reindex(gdf_points.index, fill_value=0).values / gdf_points["buffer"].area
    
#     gdf_points.drop(columns=["buffer"], inplace=True)
#     return gdf_points

# # Example usage:
# # Assuming train_gdf and test_gdf are your GeoDataFrames in EPSG:2263:
# train_gdf_intersection = compute_intersection_density(train_gdf, buffer_m=1500)
# test_gdf_intersection = compute_intersection_density(test_gdf, buffer_m=1500)

In [ ]:
# test_gdf_intersection.nunique()

In [ ]:
# !pip install rasterio
# !wget https://s3.amazonaws.com/elevation-tiles-prod/skadi/N40/N40W074.hgt.gz
# !gunzip N40W074.hgt.gz

In [ ]:
# import rasterio
# from rasterio.transform import from_origin

# # NYC Approximate Boundaries
# bounds = (-74.3, 40.5, -73.7, 40.9)

# # Create DEM GeoTIFF from HGT file
# with rasterio.open('/kaggle/working/N40W074.hgt') as src:
#     dem_data = src.read(1)
#     transform = from_origin(src.bounds.left, src.bounds.top, src.res[0], src.res[1])
#     profile = src.profile

#     profile.update({
#         'driver': 'GTiff',
#         'height': dem_data.shape[0],
#         'width': dem_data.shape[1],
#         'transform': transform,
#         'crs': 'EPSG:4326'
#     })

#     with rasterio.open('nyc_dem.tif', 'w', **profile) as dst:
#         dst.write(dem_data, 1)


In [ ]:
# import pandas as pd
# import numpy as np
# import rasterio
# from rasterio.enums import Resampling
# from scipy.ndimage import sobel, generic_filter

# # Function to extract elevation and additional features
# def extract_elevation_features(df, dem_path='nyc_dem_2263.tif'):
#     with rasterio.open(dem_path) as dem:
#         band = dem.read(1, resampling=Resampling.bilinear)
#         nodata = dem.nodata

#         # Replace nodata values with np.nan for clean processing
#         band = np.where(band == nodata, np.nan, band)

#         # Elevation Extraction
#         def get_elevation(lat, lon):
#             try:
#                 row, col = dem.index(lon, lat)
#                 if 0 <= row < band.shape[0] and 0 <= col < band.shape[1]:
#                     return band[row, col]
#                 else:
#                     return np.nan
#             except IndexError:
#                 return np.nan

#         # Slope Calculation
#         def calculate_slope(band):
#             # Replace NaNs temporarily to avoid sqrt issues
#             safe_band = np.nan_to_num(band, nan=0)
#             dx = sobel(safe_band, axis=0, mode='constant')
#             dy = sobel(safe_band, axis=1, mode='constant')
#             slope = np.sqrt(dx**2 + dy**2)
#             slope[np.isnan(band)] = np.nan  # Restore NaNs
#             return slope

#         # Aspect Calculation
#         def calculate_aspect(dx, dy):
#             aspect = np.arctan2(-dx, dy)
#             aspect = np.degrees(aspect)
#             aspect = np.where(aspect < 0, 360 + aspect, aspect)
#             return np.where(np.isnan(dx) | np.isnan(dy), np.nan, aspect)

#         # Ruggedness Calculation
#         def calculate_ruggedness(band, window=3):
#             return generic_filter(band, np.nanstd, size=window)

#         # Apply elevation
#         df['elevation'] = df.apply(lambda x: get_elevation(x['Latitude'], x['Longitude']), axis=1)

#         # Derive slope, aspect, ruggedness
#         slope = calculate_slope(band)
#         dx = sobel(np.nan_to_num(band, nan=0), axis=0, mode='constant')
#         dy = sobel(np.nan_to_num(band, nan=0), axis=1, mode='constant')
#         aspect = calculate_aspect(dx, dy)
#         ruggedness = calculate_ruggedness(band)

#         # Map features to coordinates
#         def map_to_coords(lat, lon, feature_band):
#             try:
#                 row, col = dem.index(lon, lat)
#                 if 0 <= row < feature_band.shape[0] and 0 <= col < feature_band.shape[1]:
#                     return feature_band[row, col]
#                 else:
#                     return np.nan
#             except IndexError:
#                 return np.nan

#         df['slope'] = df.apply(lambda x: map_to_coords(x['Latitude'], x['Longitude'], slope), axis=1)
#         df['aspect'] = df.apply(lambda x: map_to_coords(x['Latitude'], x['Longitude'], aspect), axis=1)
#         df['ruggedness'] = df.apply(lambda x: map_to_coords(x['Latitude'], x['Longitude'], ruggedness), axis=1)

#     return df

# # Apply to train and test sets
# train_df = extract_elevation_features(train_df.iloc[:20])
# test_df = extract_elevation_features(test_df.iloc[:20])

# # Check results
# print(train_df[['Latitude', 'Longitude', 'elevation', 'slope', 'aspect', 'ruggedness']].head())
# print(test_df[['Latitude', 'Longitude', 'elevation', 'slope', 'aspect', 'ruggedness']].head())


In [ ]:
# train_df.isnull().sum()